# research

> This notebook contains code for exploring existing research

In [18]:
#| default_exp research?

In [19]:
#| hide
from nbdev.showdoc import *

# Survey of Existing Research
This notebook contains a survey of existing research of methods to do object detection and classification.

## Building a subset of the data

In [20]:
!pip install -e ../

Obtaining file:///Users/deman/Dev/SpringboardCapstone/dog_breed_id
  Preparing metadata (setup.py) ... done
  Attempting uninstall: dog_breed_id
    Found existing installation: dog_breed_id 0.0.1
    Uninstalling dog_breed_id-0.0.1:
      Successfully uninstalled dog_breed_id-0.0.1
  Running setup.py develop for dog_breed_id


In [21]:
from dog_breed_id.data_preprocessing import read_csv_with_array_columns

In [22]:
#| export
import cv2
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
from miniai.datasets import show_images
import shutil

In [23]:
df = read_csv_with_array_columns('../data/dogs.csv', ['bboxes'])
subset = df.sample(frac=0.1)

def copy_files(path):
    target_path = str(path).replace('data', 'datasubset')
    folder = Path(target_path).parent
    folder.mkdir(exist_ok=True, parents=True)
    shutil.copy(path, target_path)
    return

In [27]:
if not Path('../datasubset').exists():
    subset['image'].apply(copy_files)
    subset.to_csv('../datasubset/dogs.csv')

## Mask RCNN
Mask RCNN is from the RCNN family of object detectors. Add more here

In [16]:
#| export
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F

In [28]:
#| hide
import nbdev; nbdev.nbdev_export()